In [2]:
import numpy as np 
import random, operator, math
import pandas as pd 
import matplotlib.pyplot as plt

In [3]:
class Projection:

    def __init__(self, type, c):

        assert c > 0
        self.type = type
        self.c = c

        if type == 'exponential':
            assert c <= .01
            def equation(x):  return c * math.pow(x,2)
        elif type == 'logistic': 
            assert c <= 50
            def equation(x): return c * math.log10(x)
        elif type == 'linear':
            assert c <= 1
            def equation(x): return c * x
        else: raise Exception('Invalid type')

        self.equation = equation

    def __repr__(self):
        if self.type == 'exponential':
            return str(self.c) + 't^2'
        elif self.type == 'logistic':
            return str(self.c) + 'log10(t)'
        else:
            return str(self.c) + 't'

exponential = Projection('exponential', .01)
assert exponential.equation(1) == .01
assert round(exponential.equation(50), 0) == 25
assert exponential.equation(100) == 100

logistic = Projection('logistic', 10)
assert logistic.equation(1) == 0
assert round(logistic.equation(50), 0) == 17
assert logistic.equation(100) == 20

linear = Projection('linear', .5)
assert linear.equation(1) == .5
assert linear.equation(50) == 25
assert linear.equation(100) == 50

In [14]:
class Asset:

    def __init__(self, id, risk, principal, projection):

       self.id = id
       self.risk = risk
       self.principal = principal
       self.projection = projection

    def __repr__(self):
       
       return ('{id: ' + str(self.id) + 
               ', risk: ' + str(self.risk) + 
               ', principal: ' + str(self.principal) +
               ', projection: ' + str(self.projection) + '}')

    def valuation(self, current, target, risk_tol):
        subjective_risk = pow(self.risk, 1 + risk_tol)
        total_risk = subjective_risk * (target - current) / 100
        return self.principal * self.projection.equation(target) * (1 - total_risk)


penny = Asset(id = 0,
              risk = .8,
              principal = 100,
              projection = exponential)

stock = Asset(id = 1,
              risk = .5,
              principal = 100,
              projection = linear)

bond = Asset(id = 2,
             risk = .2,
             principal = 100,
             projection = logistic)

print(penny)
print(stock)
print(bond)


{id: 0, risk: 0.8, principal: 100, projection: 0.01t^2}
{id: 1, risk: 0.5, principal: 100, projection: 0.5t}
{id: 2, risk: 0.2, principal: 100, projection: 10log10(t)}


In [17]:
class Agent:

    def __init__(self, risk_tol, horizon, money):

        self.risk_tol = risk_tol
        self.horizon = horizon
        self.assets = {}
        self.money = money

    def __repr__(self):

        return ('{risk_tol: ' + str(self.risk_tol) + 
                ', horizon: ' + str(self.horizon) +
                ', money: ' + str(self.money) + '}')
    
    def add_asset(self, asset):

        self.assets[asset.id] = asset

    def remove_asset(self, id):

        self.assets.pop(id)

    def buy(self, asset, price):

        self.assets[asset.id] = asset
        self.money = self.money - price

    def sell(self, asset, price):

        self.assets.pop(asset.id)
        self.money = self.money + price
    
    def evaluate(self, id, current):
        return self.assets[id].valuation(current, self.horizon, self.risk_tol)

a = Agent(risk_tol = .9,
          horizon = 100,
          money = 200)

a.add_asset(penny)
a.add_asset(stock)
a.add_asset(bond)

b = Agent(risk_tol = .2,
          horizon = 30,
          money = 200)

b.add_asset(penny)
b.add_asset(stock)
b.add_asset(bond)

print(a)
print(a.assets)
print(b.assets)

print('a penny value:', a.evaluate(0, 0))
print('a stock value:', a.evaluate(1, 0))
print('a bond value:', a.evaluate(2, 0))

print('b penny value:', b.evaluate(0, 0))
print('b stock value:', b.evaluate(1, 0))
print('b bond value:', b.evaluate(2, 0))
        

{risk_tol: 0.9, horizon: 100, money: 200}
{0: {id: 0, risk: 0.8, principal: 100, projection: 0.01t^2}, 1: {id: 1, risk: 0.5, principal: 100, projection: 0.5t}, 2: {id: 2, risk: 0.2, principal: 100, projection: 10log10(t)}}
{0: {id: 0, risk: 0.8, principal: 100, projection: 0.01t^2}, 1: {id: 1, risk: 0.5, principal: 100, projection: 0.5t}, 2: {id: 2, risk: 0.2, principal: 100, projection: 10log10(t)}}
a penny value: 3455.5828315931326
a stock value: 3660.2831718296334
a bond value: 1906.0304845529586
b penny value: 693.427860045352
b stock value: 1304.126123258372
b bond value: 1412.8860079472488
